In [13]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import numpy as np
from scipy.stats import norm

#Functions to calculate BS price for European C & P
def Black_Scholes_put_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    

def Black_Scholes_call_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    
#AD price function:

def arrow_debreu_price(p, R, prev_lambda):
    return (p / R) * prev_lambda

#Implied Volatility surface function 

def imp_volatility(K, delta_t): #Updated from example in slides to account for ∆t
    return (2.772 - 0.03797 * K + 0.0002019 * K**2 - (3.418 * 10**(-7)) * K**3) * (np.sqrt(delta_t))


#Interest rate: 
def R(r, delta_t):
    return np.exp(r * delta_t)

#Function for RN-Probability:
def rn_prob(R, d, u):
    return (R-d) / (u-d)

In [12]:
def volatility_tree(S0=149.86, r=0.03, delta_t= (37/365) / 4, time_steps=4):
    stock_price_tree = np.zeros((time_steps + 1, time_steps + 1)) #Stock price tree
    AD_tree = np.zeros((time_steps + 1, time_steps + 1)) #Arrow Debreu price tree
    probability_tree = np.zeros((time_steps + 1, time_steps + 1)) #risk-neutral probability tree
    
    
    #Initialize stock price at root node:
    stock_price_tree[0,0] = S0
    AD_tree[0,0] = 1 # λ(0,0) = 1 
    
    #Iterating through time steps:
    for n in range(1, time_steps + 1):
        if n == 1:
            K = stock_price_tree[n-1, 0]
            imp_vol = imp_volatility(K, delta_t)
            #S(1,0) and S(1,1) unknown --> Case3
            #Need V_put(0,0):
            time_0_BS_put_price = Black_Scholes_put_price(S = S0, K = K, T = n * delta_t, r = r, sigma=imp_vol)
            V_put_00 = time_0_BS_put_price / AD_tree[0,0]
            #Now need to calculate u(0,0) & d(0,0):
            u_00 = (S0 + V_put_00) / ((S0 / R(r, delta_t)) - V_put_00)
            d_00 = 1 / u_00
            
            stock_price_tree[1,1] = u_00 * stock_price_tree[n - 1, 0] #S(1,1)
            stock_price_tree[1,0] = d_00 * stock_price_tree[n - 1, 0] #S(1,0)
            
            #Now need to check that S(1,0) < S(0,0)R(0,0) < S(1,1)
#             if (stock_price_tree[1,0] < stock_price_tree[0,0] * R(r, delta_t)) & (stock_price_tree[0,0] * R(r, delta_t) < stock_price_tree[1,1]):
#                 print('Condition met')
#             else:
#                  print('Condition not met')
            
            #Calculating RN Probabillity:
            p_00 = rn_prob(R = R(r, delta_t), d = d_00, u = u_00)
            probability_tree[0,0] = p_00
            #Calculating AD-prices:
            AD_tree[1,1] = arrow_debreu_price(p = p_00, R = R(r, delta_t), prev_lambda = AD_tree[0,0])
            
            AD_tree[1,0] = arrow_debreu_price(p = 1 - p_00, R = R(r, delta_t), prev_lambda = AD_tree[0,0])
            
            
            #Done with step 1
            
        if n % 2 == 0: #i.e. even time steps where we'll start at middle node, set=90 and expand out using case1, case2
            m = n // 2 # will be used as a base index for j from middle node
            
            stock_price_tree[n, m] = stock_price_tree[0,0] #set middle node = 90
            
            for j in range(m + 1, n + 1): #n = 2 --> j = 2, n=4 --> j = 3,4 ie iterating up tree from middle node & using Case 2
                K_n = stock_price_tree[n-1, j-1]
                imp_vol = imp_volatility(K = K_n, delta_t = delta_t)
                        
                #Need to calc BS Call price:
                time_0_BS_call_price = Black_Scholes_call_price(S = S0, K = K_n, sigma = imp_vol, T = n * delta_t, r = r)
            
                #Use AD Securities to get Value of call
                if (n == 2) or (n == 4 and j == 4):
                    V_call = time_0_BS_call_price / AD_tree[n-1, j-1]
                                                
                elif n == 4:
                    sum_calls = AD_tree[n-1, j] * (1 / R(r, delta_t)) * (stock_price_tree[n-1, j]  * R(r, delta_t) - stock_price_tree[n-1, j-1]) #formula on slide 62
                    V_call = (time_0_BS_call_price - sum_calls) / AD_tree[n-1, j-1] #For S(4,3)
                    
                    
                #Case 2 formula to calculate stock price
                stock_price_tree[n, j] = (V_call * stock_price_tree[n, j-1] + stock_price_tree[n-1, j-1] * ((stock_price_tree[n, j-1]/R(r, delta_t)) - stock_price_tree[n-1, j-1])) / (V_call + (stock_price_tree[n, j-1]/R(r, delta_t)) - stock_price_tree[n-1, j-1]) #HERE
                
                           
            
            for j in range(m - 1, -1, -1): # n= 2 --> j = 0, n = 4 --> j = 1,0 ie iterating down from middle and using case 1
                K_n = stock_price_tree[n-1, j]
                imp_vol = imp_volatility(K = K_n, delta_t = delta_t)
                        
                #BS Put Price:
                time_0_BS_put_price = Black_Scholes_put_price(K=K_n, S = S0, T = n * delta_t, r = r, sigma=imp_vol)
                        
                #Calculate V_put using Arrow-Debreu Securities: NOTE: at n = 4, j=1 will need to calc a sum of AD prices (See slides 59-60)
                if (n == 2) or (n == 4 and j == 0): 
                    V_put = time_0_BS_put_price / AD_tree[n-1, j]
                elif (n == 4) and (j == 1):
                    sum_puts = (AD_tree[n-1, j-1] / R(r, delta_t)) * (stock_price_tree[n-1, j] - stock_price_tree[n-1, j-1] * R(r, delta_t)) #Formula from slide 60 for Σp
                    V_put = (time_0_BS_put_price - sum_puts) / AD_tree[n-1, j]
                    
                    
                    
                #Using Case 1 Formula to calculate Stock price:
                stock_price_tree[n, j] = (V_put * stock_price_tree[n,j+1] + stock_price_tree[n-1, j] * (stock_price_tree[n-1, j] - (stock_price_tree[n,j+1]/ R(r, delta_t)))) / (V_put + stock_price_tree[n-1, j] - (stock_price_tree[n,j+1]/R(r, delta_t)))
                
            #Calculating risk-neutral probabilities at end of time n=2
            for j in range(0,n):#n=2 here
                probability_tree[n-1, j] = (R(r, delta_t) * stock_price_tree[n-1, j] - stock_price_tree[n, j]) / (stock_price_tree[n, j+1] - stock_price_tree[n, j])
            
            #Calculating AD prices:
            for j in range(0, n+1):
                if j == n / 2:
                    AD_tree[n, j] = arrow_debreu_price(p = probability_tree[n-1, j-1], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j-1]) + arrow_debreu_price(p = 1 - probability_tree[n-1, j], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j])
                elif j == 0:
                    AD_tree[n, j] = arrow_debreu_price(p = 1 - probability_tree[n-1, j], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j])
                else:
                    AD_tree[n, j] = arrow_debreu_price(p = probability_tree[n-1, j-1], R = R(r, delta_t),  prev_lambda = AD_tree[n-1, j-1])

                   
        
        
        if (n % 2 == 1) and (n > 1): #Odd numbered steps after n = 1 (ie n = 3 here)
            
            m = n // 2
            for j in range(m, m+1):#Case 3 to get S(3,1) and S(3,2), just need m = j = 1 then we will iterate up and down
                K_n = stock_price_tree[n-1, j]
                imp_vol = imp_volatility(K = K_n, delta_t = delta_t)
                time_0_BS_put_price = Black_Scholes_put_price(S = S0, T = n * delta_t, r = r, sigma = imp_vol, K = K_n)
                
                #Use Arrow-Debreu securities:
                sum_puts = AD_tree[n-1, j-1] * (1 / R(r, delta_t)) * (stock_price_tree[n-1, j] - stock_price_tree[n-1, j-1] * R(r, delta_t))
                V_put = (time_0_BS_put_price - sum_puts) / AD_tree[n-1,j]
                
                #Now calculate u and d to get S(3,1) & S(3,2)
                u_ = (stock_price_tree[n-1, j] + V_put) / ((stock_price_tree[n-1, j] / R(r, delta_t)) - V_put)
                
                
                stock_price_tree[n,j+1] = stock_price_tree[n-1, j] * u_ #S(3,2)
                stock_price_tree[n, j] = stock_price_tree[n-1, j] / u_ #S(3,1)
                
                #Now iterate j down from m until 0 and use Case 1:
            for j in range(m-1, -1, -1): #m = j = 0 for n = 3
                K_n  = stock_price_tree[n-1, j]
                imp_vol = imp_volatility(K = K_n, delta_t = delta_t)
                time_0_BS_put_price = Black_Scholes_put_price(S = S0, T = n * delta_t, r = r, sigma = imp_vol, K = K_n)
                V_put = time_0_BS_put_price / AD_tree[n-1, j]
                stock_price_tree[n, j] = (V_put * stock_price_tree[n, j+1] + stock_price_tree[n-1, j] * (stock_price_tree[n-1, j] - (stock_price_tree[n, j+1] / R(r, delta_t)))) / (V_put + stock_price_tree[n-1, j] - (stock_price_tree[n, j+1] / R(r, delta_t)))
                    
                    
            for j in range(m+2, n+1): #Iterating up from 2 nodes calculated using Case 3 until top of tree (at n=3, j = 3 and use Case 2)
                K_n = stock_price_tree[n-1, j-1]
                imp_vol = imp_volatility(K = K_n, delta_t = delta_t)
                    
                time_0_BS_call_price = Black_Scholes_call_price(S = S0, K = K_n, T = n * delta_t, sigma = imp_vol, r = r)
                V_call = time_0_BS_call_price / AD_tree[n-1, j-1]
                    
                stock_price_tree[n, j] = (V_call * stock_price_tree[n, j-1] + stock_price_tree[n-1, j-1] * ((stock_price_tree[n, j-1]/R(r, delta_t)) - stock_price_tree[n-1, j-1])) / (V_call + (stock_price_tree[n, j-1]/R(r, delta_t) - stock_price_tree[n-1, j-1]))
                
            #Now calculate RISK NEUTRAL PROBABILITIES:
            for j in range(0, n):
                probability_tree[n-1, j] = (R(r, delta_t) * stock_price_tree[n-1, j] - stock_price_tree[n, j]) / (stock_price_tree[n, j+1] - stock_price_tree[n, j])
            
            
            
            #AD Tree calculations:
            for j in range(0, n+1): #n = 3, j = 0, 1, 2, 3
                if j == n:
                    AD_tree[n, j] = arrow_debreu_price(p = probability_tree[n-1, j-1], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j-1])
                elif j == 0:
                    AD_tree[n, j] = arrow_debreu_price(p = 1 - probability_tree[n-1, j], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j])
                else: #If j != n, need sum of AD Prices from connected nodes: (here n = 3, j = 1 and 2)
                    AD_tree[n, j] = arrow_debreu_price(p = 1 - probability_tree[n-1, j], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j]) + arrow_debreu_price(p = probability_tree[n-1, j-1], R = R(r, delta_t), prev_lambda = AD_tree[n-1, j-1])
        
                        
                        
                        
                        

    return stock_price_tree, AD_tree, probability_tree
    
np.set_printoptions(linewidth=np.inf)
print(f' Stock Price Tree:\n {volatility_tree()[0]}')
print()
print(f' Arrow Debreu Tree:\n {volatility_tree()[1]}')
print()
print(f' Probability Tree:\n {volatility_tree()[2]}')
print()

 Stock Price Tree:
 [[149.86         0.           0.           0.           0.        ]
 [148.45242088 151.28092535   0.           0.           0.        ]
 [145.92325053 149.86       153.65158944   0.           0.        ]
 [144.78684484 148.23593653 151.50185661 155.20235145   0.        ]
 [142.37654117 146.37725459 149.86       153.39264581 157.51288667]]

 Arrow Debreu Tree:
 [[1.         0.         0.         0.         0.        ]
 [0.46171188 0.53752814 0.         0.         0.        ]
 [0.15172695 0.62916491 0.21758874 0.         0.        ]
 [0.09678017 0.34894677 0.41882058 0.13317425 0.        ]
 [0.03578208 0.06092454 0.4077186  0.20815655 0.0622612 ]]

 Probability Tree:
 [[0.53793696 0.         0.         0.         0.        ]
 [0.6711318  0.40510294 0.         0.         0.        ]
 [0.3616574  0.53217513 0.61251121 0.         0.        ]
 [0.62999344 0.56605479 0.49738451 0.46787236 0.        ]
 [0.         0.         0.         0.         0.        ]]



## 1b:

In [15]:
#Carrying over from results (ie coimputed stock and probability trees) of part a:

stock_price_tree = np.array([[149.86      ,   0.        ,   0.        ,   0.        ,   0.        ],
                             [148.45242088, 151.28092535,   0.        ,   0.        ,   0.        ],
                             [145.92325053, 149.86      , 153.65158944,   0.        ,   0.        ],
                             [144.78684484, 148.23593653, 151.50185661, 155.20235145,   0.        ],
                             [142.37654117, 146.37725459, 149.86      , 153.39264581, 157.51288667]])

probability_tree = np.array([[0.53793696, 0.        , 0.        , 0.        , 0.        ],
                            [0.6711318 , 0.40510294, 0.        , 0.        , 0.        ],
                            [0.3616574 , 0.53217513, 0.61251121, 0.        , 0.        ],
                            [0.62999344, 0.56605479, 0.49738451, 0.46787236, 0.        ],
                            [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [16]:
def American_Put_Pricer(stock_price_tree, probability_tree, K, r, N):
   
    #R used to discount:
    delta_t = 1 / N
    R = np.exp(-r * delta_t)
    
    #Initializing option value tree with same dimensions as previously calculated stock price tree:
    option_value_tree = np.zeros_like(stock_price_tree)

    #Payoffs @ time N (using maximum function since this option is American)
    option_value_tree[N, :] = np.maximum(K - stock_price_tree[N, :], 0)  #might need to change back to np.maximum(K - stock_price_tree[:, N], 0)

    #Backward Induction:
    for n in range(N - 1, -1, -1): #starting at N = 3 here and working our way back to time 0
        for j in range(n + 1): #iterating through number heads at each time step
            
            #value discounting back and using risk-neutral probabilities
            discount_value = R * (probability_tree[n, j] * option_value_tree[n+1, j] + 
                                      (1 - probability_tree[n, j]) * option_value_tree[n+1, j+1])
            
            # Intrinsic value at node (n, j) (i.e. value if exercised)
            intrinsic_value = max(K - stock_price_tree[n, j], 0)
            
            
            option_value_tree[n, j] = max(discount_value, intrinsic_value)

    
    return option_value_tree#[0, 0] #i.e price of the put



tree = American_Put_Pricer(stock_price_tree = stock_price_tree, probability_tree = probability_tree, K = 150, r = 0.03, N = 4)
print('Option Value Tree:', '\n')
print(tree, '\n')
print('------------------------------------------------------------\n')
print(f"Time 0 price of American put: {tree[0,0]}")


Option Value Tree: 

[[1.86981208 0.         0.         0.         0.        ]
 [3.08737807 0.48277744 0.         0.         0.        ]
 [4.07674947 1.13901023 0.0420165  0.         0.        ]
 [6.0972672  2.09564827 0.06911353 0.         0.        ]
 [7.62345883 3.62274541 0.14       0.         0.        ]] 

------------------------------------------------------------

Time 0 price of American put: 1.8698120750987757
